# Косенко Никита М24-514
## Лабораторная работа №3, вариант 6

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Подключаем данные

In [2]:
data = pd.read_csv('data_ml_v3-06.csv')
X = data['x']
y = data['y']

data

,x,y
0,-0.60604,-1.179300
1,-0.35888,-2.347900
2,-2.05920,1.811500
3,-1.04380,1.678300
4,-1.11560,-0.018185
...,...,...
295,-2.53810,4.285800
296,-0.15471,-0.455390
297,2.01020,-1.383500
298,-0.18364,2.537300


### Задание 1
Построить регрессограммы для различного числа интервалов группировки:

а) 3

б) 10

в) 50

Построить график зависимости MSE непараметрической регрессии на обучающей, 
валидационной и тестовой выборках от числа интервалов группировки. Разбиение на обучающую, валидационную и тестовую выборки – holdout в отношении 60/30/10.

### Задание 2
Построить графики зависимости MSE ядерной регрессии на обучающей, валидационной и тестовой выборках от ширины окна для:

а) прямоугольного окна

б) гауссова окна

в) окна  Епанечникова

г) треугольного окна

Определить оптимальную ширину каждого окна. Оптимальные  значения гиперпараметров обучения определять по значению 
ошибки на валидационной выборке.

### Задание 3
Визуализировать графики ядерной функции регрессии, полученной с использованием каждого вида окна из задания 2.

Ширину окна  задать:

i)  равной  соответствующему  оптимальному  значению,  полученному  в задании 2

ii) значению, меньшему оптимального

iii) значению, большему оптимального 

### Задание 4
Построить  графики зависимости дисперсии и смещения регрессионной модели для 
нескольких фиксированных значений *x* от ширины окна для каждого вида окна из задания 2.

### Задание 5
Построить график зависимости MSE KNN-регрессии от числа ближайших соседей K на 
обучающей и тестовой выборках. Определить оптимальное число ближайших соседей и 
визуализировать  графики  KNN-регрессии для различных значений  K:

а) равному оптимальному

б) меньшего оптимального

в) большего оптимального

Оптимальные  значения гиперпараметров обучения  определять по значению ошибки на валидационной выборке.

### Задание 6
Сделать выводы о влиянии вида и ширины окна на качество ядерной регрессионной модели.